<h3>Creating the RDD's</h3>

<b>There are three ways to create RDDs in Spark</b>
<ul>
       <li>Parallelizing via collections in driver program.</li>
       <li>Creating a dataset in an external storage system (e.g. HDFS, HBase, and Shared FS).</li>
       <li>Creating RDD from existing RDDs.</li>
</ul>

<h3>Parallelizing via collections in driver program</h3>

In [1]:
# Packages that must be Imported
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
sparkConf = SparkConf( ) \
 .setAppName("Checking") \
 .setMaster("local") 
sc = SparkContext(conf=sparkConf)

In [3]:
spark = SparkSession.builder\
        .appName("WordCount")\
        .master("local[3]")\
        .getOrCreate()
# spark.sparkContext( ) 

In [4]:
data = [1,2,3,4,5,6,7,8,9,10,11,12]
rdd = sc.parallelize(data)
rdd.collect( )
print (rdd.take(20))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


<h3>What is parallelized in spark</h3>
<p>Parallelize is a <b>method to create an RDD from an existing collection (For e.g Array)</b>
    present in the driver.<br> The elements present in the collection are copied to form a distributed dataset on which we can operate on in parallel.</p>

<h3>Create RDD with partition</h3>

In [5]:
data = [1,2,3,4,5,6,7,8,9,10,11,12]
rdd = sc.parallelize(data)
print("Initial Partition Count:"+str(rdd.getNumPartitions()))
partition_data = [1,2,3,4,5,6,7,8,9,10,11,12]
partition_rdd = spark.sparkContext.parallelize(partition_data, 5)
print(partition_rdd.collect())
print(partition_rdd.glom().collect())
print("After changing Partition Count:"+str(partition_rdd.getNumPartitions()))

Initial Partition Count:1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10, 11, 12]]
After changing Partition Count:5


### what is the Glom usef for ?
<p>Return an RDD created by coalescing all elements within each partition into a list.</p>

<h3>Create RDD from external text file</h3>

In [6]:
readFile = sc.textFile("file:///home/saif/LFS/datasets/emp.txt")
readFile.collect() 

['id,name,city', '101,saif,mumbai', '102,mitali,pune', '103,ram,balewadi']

<h3>Creating RDD from existing RDD</h3>

In [7]:
print("Initial Partition Count:"+str(readFile.getNumPartitions()))
repartition_Rdd = readFile.repartition(4)
print("Re-partition count:"+str(repartition_Rdd.getNumPartitions()))
coalesce_Rdd = readFile.repartition(3)
print("Re-partition count:"+str(coalesce_Rdd.getNumPartitions())) 

Initial Partition Count:1
Re-partition count:4
Re-partition count:3


<h2>Map</h2>

In [8]:
a = sc.parallelize([1,2,3,4,5])
result = a.map(lambda x: (x, x+2))
result.collect()
for element in result.collect():
    print(element, end="")

(1, 3)(2, 4)(3, 5)(4, 6)(5, 7)

### flatMap 
<p>Returns a new RDD by first applying a function to all elements of this RDD, and then
flattening the results.</p>

In [9]:
a = sc.parallelize([1,2,3,4,5])
result = a.flatMap(lambda x: (x, x**2))
result.collect() 

[1, 1, 2, 4, 3, 9, 4, 16, 5, 25]

### filter

In [10]:
a = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
result = a.filter(lambda x: x % 2 == 0)
print(result.collect()) 

[2, 4, 6, 8, 10]


### union

In [11]:
a = sc.parallelize([1,2,3,4,5])
b = sc.parallelize([1,2,2,3,3])
a.union(b).collect() 

[1, 2, 3, 4, 5, 1, 2, 2, 3, 3]

### intersection

In [12]:
a = sc.parallelize([1,2,3,4,5])
b = sc.parallelize([1,2,2,3,3])
a.intersection(b).collect()

[2, 1, 3]

### distinct

In [13]:
a = sc.parallelize([1,2,2,3,4,4,4,5])
a.distinct( ).collect() 

[1, 2, 3, 4, 5]

### groupByKey

In [14]:
x = sc.parallelize([('A', 2), ('B', 1), ('B', 5), ('A', 1), ('B', 10)])
result = x.groupByKey()
print(result.collect())
for j in result.collect():
    for i in j[1]:
        print(i) 

[('A', <pyspark.resultiterable.ResultIterable object at 0x7fa4f7a377c0>), ('B', <pyspark.resultiterable.ResultIterable object at 0x7fa4f7acaee0>)]
2
1
1
5
10


<img src="https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3D81l3yBTUWAA&psig=AOvVaw3HJKc_N3SDAiXYf2CftEtp&ust=1642398686918000&source=images&cd=vfe&ved=0CAgQjRxqFwoTCMiN_ITKtfUCFQAAAAAdAAAAABAD"></img>

### reduceByKey

In [15]:
words = sc.parallelize(["Saif", "Ram", "Mitali", "Aniket", "Ram", "Ram", "Aniket"])
wordCount = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b: a + b)
print(wordCount.collect()) 

[('Saif', 1), ('Ram', 3), ('Mitali', 1), ('Aniket', 2)]


<h3>Spark difference between reduceByKey vs. groupByKey vs. aggregateByKey vs. combineByKey</h3>
<a href="https://stackoverflow.com/questions/43364432/spark-difference-between-reducebykey-vs-groupbykey-vs-aggregatebykey-vs-combi#:~:text=groupByKey%20can%20cause%20out%20of,collected%20on%20the%20reduced%20workers.&text=Data%20are%20combined%20at%20each,with%20the%20exact%20same%20type" target="_blank">LINK</a>
<h3>Is groupByKey ever preferred over reduceByKey ?</h3>
<a href="https://stackoverflow.com/questions/33221713/is-groupbykey-ever-preferred-over-reducebykey" target="_blank">Link</a>

### sortByKey

In [16]:
words = sc.parallelize(["Saif", "Ram", "Mitali", "Aniket", "Ram", "Ram", "Aniket"])
wordCount = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b: a + b)
print(wordCount.sortByKey().collect())
print(wordCount.sortByKey(False).collect())

[('Aniket', 2), ('Mitali', 1), ('Ram', 3), ('Saif', 1)]
[('Saif', 1), ('Ram', 3), ('Mitali', 1), ('Aniket', 2)]


<h3>Join</h3>

In [18]:
a = sc.parallelize([('C', 4), ('B', 3), ('A', 2), ('A', 1)])
b = sc.parallelize([('A', 8), ('B', 7), ('A', 6), ('D', 5)])
print(a.join(b).collect()) 

[('B', (3, 7)), ('A', (2, 8)), ('A', (2, 6)), ('A', (1, 8)), ('A', (1, 6))]


<h3>Types of joins:</h3>
<ul>
    <li>join</li>
    <li>leftOuterJoin</li>
    <li>rightOuterJoin</li>
    <li>fullOuterJoin</li>
    <li>Cartesian</li>
</ul>

<h3>Coalesce</h3>
<ul>
<li>In coalesce ( ) we use existing partition so that less data is shuffled.<br> To avoid full
shuffling of data we use coalesce ( ) function.<br> Using this we can reduce the number
of the partition.</li>
<li>Creates unequal sized partitions.</li>
</ul>

<h3>Repartition</h3>
<ul>
    <li>Used to increase or decrease the number of partitions.</li>
    <li>A network shuffle will be triggered which can increase data movement.</li>
    <li>Creates equal sized partitions.</li>
<ul>

In [20]:
a = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
b = a.getNumPartitions()
print(b)
c = a.glom().collect()
print(c) 
d = a.coalesce(1)
print("After Coalesce: "+str(d.getNumPartitions()))
e = a.repartition(5)
print("After Repartition: "+str(e.getNumPartitions())) 

1
[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
After Coalesce: 1
After Repartition: 5


<h2>Action Operations</h2>
<ul>
    <li>count</li>
    <li>take</li>
    <li>collect</li>
    <li>top</li>
    <li>countByValue</li>
    <li>countByKey</li>
    <li>reduce</li>
</ul>


In [22]:
a = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
a.count( ) 

10

In [23]:
a.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [24]:
a.take(5) 

[1, 2, 3, 4, 5]

In [25]:
a.top(4) 

[10, 9, 8, 7]

In [26]:
a = sc.parallelize(["Saif", "Mitali", "Ram", "Ram", "Ram", "Mitali"])
a.countByValue()
a.countByValue().keys()
a.countByValue().values() 

dict_values([1, 2, 3])

In [27]:
x = sc.parallelize([('A', 2), ('A', 1), ('C',1), ('B',5)])
x.countByKey()
x.countByKey().values() 

dict_values([2, 1, 1])

In [28]:
a = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
a.reduce(lambda a, b: a + b) 

55